<a href="https://colab.research.google.com/github/Navisekhar/mobiux_assignment/blob/main/Mobiux_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from collections import defaultdict

url = 'https://mobiux.in/assignment/sales-data.txt'
response = requests.get(url)

if response.status_code == 200:
    with open('sales_data.txt', 'w') as file:
        file.write(response.text)
else:
    print(f"Failed to download the file. HTTP Status Code: {response.status_code}")

sales_data = []
with open('sales_data.txt', 'r') as file:
    lines = file.readlines()[1:]
    for line in lines:
        date, sku, unit_price, quantity, total_price = line.strip().split(',')
        sale = {
            "Date": date,
            "SKU": sku,
            "Unit Price": int(unit_price),
            "Quantity": int(quantity),
            "Total Price": int(total_price)
        }
        sales_data.append(sale)

monthly_sales = defaultdict(lambda: {"total_sales": 0, "items_sold": defaultdict(int), "revenue": defaultdict(int)})

for sale in sales_data:
    month = sale["Date"].split('-')[1]
    sku = sale["SKU"]
    quantity = sale["Quantity"]
    revenue = sale["Total Price"]

    monthly_sales[month]["total_sales"] += revenue
    monthly_sales[month]["items_sold"][sku] += quantity
    monthly_sales[month]["revenue"][sku] += revenue

popular_items = {}
revenue_items = {}
order_stats = {}

for month, data in monthly_sales.items():

    most_popular_item = max(data["items_sold"], key=data["items_sold"].get)
    most_revenue_item = max(data["revenue"], key=data["revenue"].get)

    popular_items[month] = most_popular_item
    revenue_items[month] = most_revenue_item

    if most_popular_item not in order_stats:
        order_stats[most_popular_item] = {"min": float('inf'), "max": 0, "total": 0, "count": 0}

    quantity_sold = data["items_sold"][most_popular_item]
    stats = order_stats[most_popular_item]
    stats["min"] = min(stats["min"], quantity_sold)
    stats["max"] = max(stats["max"], quantity_sold)
    stats["total"] += quantity_sold
    stats["count"] += 1

for item, stats in order_stats.items():
    stats["avg"] = stats["total"] / stats["count"]

total_sales = sum(data["total_sales"] for data in monthly_sales.values())
print("Total sales of the store:", total_sales)

print("\nMonth-wise sales totals:")
for month, data in monthly_sales.items():
    print(f"Month {month}: Total Sales = {data['total_sales']}")

print("\nMost popular item in each month:")
for month, item in popular_items.items():
    print(f"Month {month}: {item}")

print("\nItems generating most revenue in each month:")
for month, item in revenue_items.items():
    print(f"Month {month}: {item}")

print("\nMin, Max, and Avg orders for most popular items:")
for item, stats in order_stats.items():
    print(f"{item}: Min Orders = {stats['min']}, Max Orders = {stats['max']}, Avg Orders = {stats['avg']:.2f}")


Total sales of the store: 4583270

Month-wise sales totals:
Month 01: Total Sales = 1421330
Month 02: Total Sales = 1422350
Month 03: Total Sales = 1739590

Most popular item in each month:
Month 01: Hot Chocolate Fudge
Month 02: Hot Chocolate Fudge
Month 03: Hot Chocolate Fudge

Items generating most revenue in each month:
Month 01: Hot Chocolate Fudge
Month 02: Hot Chocolate Fudge
Month 03: Hot Chocolate Fudge

Min, Max, and Avg orders for most popular items:
Hot Chocolate Fudge: Min Orders = 2636, Max Orders = 3100, Avg Orders = 2803.00
